In [55]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
%matplotlib inline

In [57]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4" 

# 设置 HTTP 和 HTTPS 代理
os.environ["http_proxy"] = "***"
os.environ["https_proxy"] = "***"

# predict

## text prompt
CUDA_VISIBLE_DEVICES=3 python predict_text_prompt.py \
    --source ultralytics/assets/bus.jpg \
    --checkpoint pretrain/yoloe-v8l-seg.pt \
    --names person dog cat \
    --device cuda:0

In [58]:
import argparse
import os
from PIL import Image
import supervision as sv
from ultralytics import YOLOE

In [59]:
import matplotlib.pyplot as plt

In [61]:
source = "/root/project/research/Yolo/yoloe/test_data/000000039769.jpg"
checkpoint = "./pretrain/yoloe-v8l-seg.pt"

# 微调
# source = "/root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp"
# checkpoint = "/root/project/research/Yolo/ultralytics/runs/segment/train6/weights/best.pt"

device = "cuda:0"
# text_prompt = ["person", "dog", "cat"]
# text_prompt = ["remote control"]
# text_prompt = ["一只躺在图片左边的猫"]
# text_prompt = ["白色的遥控器"]
# text_prompt = ["person", "dog", "two cats", "cat", "remote control", "sofa", "chair", "teacup", "coffee"]  # 中文效果很差，label需要使用中文; 语义理解能力很差
# text_prompt = ["person", "dog", "two cats", "the cat on the far left", "remote control", "sofa"]
# text_prompt = ["person", "dog", "two cats", "remote control", "sofa"]
# text_prompt = ["coffee cup"]

text_prompt = ["person", "dog", "two cats", "cat", "remote control", "sofa", "chair", "teacup", "coffee", "sliding sleeve"] 

output = "/root/project/research/Yolo/yoloe/test_data_out/"
os.makedirs(output, exist_ok=True)

image = Image.open(source).convert("RGB")

# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(image, cmap="gray") 

In [62]:
model = YOLOE(checkpoint)
model.to(device)
print(f" model.get_text_pe(text_prompt): { model.get_text_pe(text_prompt).shape}")
model.set_classes(text_prompt, model.get_text_pe(text_prompt)) # model.get_text_pe(text_prompt).shape, torch.Size([1, x, 512])
# results = model.predict(image, conf=0.25, verbose=True)
results = model.predict(image, verbose=True)

Build text model mobileclip:blt
 model.get_text_pe(text_prompt): torch.Size([1, 10, 512])
Build text model mobileclip:blt

0: 640x640 2 cats, 2 remote controls, 1 sofa, 9.9ms
Speed: 1.2ms preprocess, 9.9ms inference, 12.6ms postprocess per image at shape (1, 3, 640, 640)


In [64]:
detections = sv.Detections.from_ultralytics(results[0])
# print(f"detections: {detections}")

resolution_wh = image.size

thickness = sv.calculate_optimal_line_thickness(resolution_wh=resolution_wh)
text_scale = sv.calculate_optimal_text_scale(resolution_wh=resolution_wh)

labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence in zip(detections["class_name"], detections.confidence)
]

annotated_image = image.copy()

annotated_image = sv.MaskAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    opacity=0.4
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.BoxAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    thickness=thickness
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.LabelAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    text_scale=text_scale,
    smart_position=True
).annotate(scene=annotated_image, detections=detections, labels=labels)

print(f"annotated_image: {annotated_image.size}")
# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(annotated_image, cmap="gray")

base, ext = os.path.splitext(source)
path, filename = os.path.split(source)
# print(f"base: {path}, ext: {filename}")
output_name = os.path.join(output, "re_" + filename)
# print(f"output_name: {output_name}")
annotated_image.save(output_name)

annotated_image: (640, 480)


## train pe result

`text_prompt = ['bracket component', 'sliding sleeve', 'elastic bridge', 'fastening screw', 'fastening nut', 'small adjustment block', 'large adjustment block']` 数量与微调的 数据类别必须一致，否则会报错，相当于回到闭集


```python
from ultralytics import YOLOE

# 加载训练好的模型
import torch
pe_data = torch.load("custom-pe.pt")
names = pe_data["names"]
print(f"names: {names}")
pe = pe_data["pe"]
print(f"pe: {pe.shape}")
model = YOLOE("/root/project/research/Yolo/ultralytics/runs/segment/train6/weights/last.pt")
model.set_classes(names, model.get_text_pe(names))

# model = YOLOE("/root/project/research/Yolo/ultralytics/runs/segment/train6/weights/best.pt")  # 或你自定义保存的路径

# 推理图片
results = model.predict(source='/root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp', save=True)

# 打印结果
for result in results:
    print(result.boxes)       # 检测框
    print(result.masks)       # 分割掩膜（如果是 segment 模型）
    print(result.probs)       # 类别概率（可选）
    
```
or

```python
from ultralytics import YOLOE

# 加载训练好的模型
import torch
model = YOLOE("/root/project/research/Yolo/ultralytics/runs/segment/train6/weights/best.pt")  # 或你自定义保存的路径
# 推理图片
results = model.predict(source='/root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp', save=True)

# 打印结果
for result in results:
    print(result.boxes)       # 检测框
    print(result.masks)       # 分割掩膜（如果是 segment 模型）
    print(result.probs)       # 类别概率（可选）
```

In [36]:
import argparse
import os
from PIL import Image
import supervision as sv
from ultralytics import YOLOE

In [37]:
import matplotlib.pyplot as plt

In [65]:
# source = "/root/project/research/Yolo/yoloe/test_data/000000039769.jpg"
# checkpoint = "./pretrain/yoloe-v8l-seg.pt"

# 微调
source = "/root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp"
checkpoint = "/root/project/research/Yolo/ultralytics/runs/segment/train7/weights/best.pt"

device = "cuda:0"
# text_prompt = ["person", "dog", "cat"]
# text_prompt = ["remote control"]
# text_prompt = ["一只躺在图片左边的猫"]
# text_prompt = ["白色的遥控器"]
# text_prompt = ["person", "dog", "two cats", "cat", "remote control", "sofa", "chair", "teacup", "coffee"]  # 中文效果很差，label需要使用中文; 语义理解能力很差
# text_prompt = ["person", "dog", "two cats", "the cat on the far left", "remote control", "sofa"]
# text_prompt = ["person", "dog", "two cats", "remote control", "sofa"]
# text_prompt = ["coffee cup"]

text_prompt = ['bracket component', 'sliding sleeve', 'elastic bridge', 'fastening screw', 'fastening nut', 'small adjustment block', 'large adjustment block']

# text_prompt = ['bracket component', 'small adjustment block', 'large adjustment block']

output = "/root/project/research/Yolo/yoloe/test_data_out/"
os.makedirs(output, exist_ok=True)

image = Image.open(source).convert("RGB")

# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(image, cmap="gray") 

In [66]:
model = YOLOE(checkpoint)
model.to(device)
print(f" model.get_text_pe(text_prompt): { model.get_text_pe(text_prompt).shape}")
model.set_classes(text_prompt, model.get_text_pe(text_prompt)) # model.get_text_pe(text_prompt).shape, torch.Size([1, x, 512])
# results = model.predict(image, conf=0.25, verbose=True)
results = model.predict(image, verbose=True)

Build text model mobileclip:blt
 model.get_text_pe(text_prompt): torch.Size([1, 7, 512])
Build text model mobileclip:blt

0: 640x640 2 bracket components, 2 sliding sleeves, 2 elastic bridges, 12.0ms
Speed: 2.3ms preprocess, 12.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


In [67]:
detections = sv.Detections.from_ultralytics(results[0])
# print(f"detections: {detections}")

resolution_wh = image.size

thickness = sv.calculate_optimal_line_thickness(resolution_wh=resolution_wh)
text_scale = sv.calculate_optimal_text_scale(resolution_wh=resolution_wh)

labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence in zip(detections["class_name"], detections.confidence)
]

annotated_image = image.copy()

annotated_image = sv.MaskAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    opacity=0.4
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.BoxAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    thickness=thickness
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.LabelAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    text_scale=text_scale,
    smart_position=True
).annotate(scene=annotated_image, detections=detections, labels=labels)

print(f"annotated_image: {annotated_image.size}")
# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(annotated_image, cmap="gray")

base, ext = os.path.splitext(source)
path, filename = os.path.split(source)
print(f"base: {path}, ext: {filename}")
output_name = os.path.join(output, "re_" + filename)
print(f"output_name: {output_name}")
annotated_image.save(output_name)

annotated_image: (3072, 2048)
base: /root/dataset/glass_data_20250317/images/train, ext: Image_20250210134520105.bmp
output_name: /root/project/research/Yolo/yoloe/test_data_out/re_Image_20250210134520105.bmp


## COCO 转 labelme json

In [52]:
import json
import os
from shutil import copyfile

# 输入输出路径
input_json = '/root/project/research/Yolo/yoloe/custom_train.json'
output_dir = '/root/dataset/converted_labelme_json/'

# 读取原始的目标检测标注 JSON 文件
with open(input_json, 'r') as f:
    data = json.load(f)

# 检查输出目录是否存在
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 创建类别映射，将 category_id 映射到 category_name
categories = {category['id']: category['name'] for category in data['categories']}

# 假设目标检测 JSON 文件包含一个图片列表，每个图片有 'file_name' 和 'annotations' 字段
for img_info in data['images']:
    img_name = img_info['file_name']
    img_id = img_info['id']
    
    # 查找该图片对应的标注
    annotations = [ann for ann in data['annotations'] if ann['image_id'] == img_id]

    # 创建 LabelMe 格式的 JSON
    labelme_json = {
        "version": "4.5.7",  # 这里可以根据实际版本修改
        "flags": {},
        "shapes": [],
        "imagePath": img_name,
        "imageData": None,
        "imageHeight": img_info['height'],
        "imageWidth": img_info['width']
    }

    # 转换目标检测框为 LabelMe 支持的格式
    for ann in annotations:
        xmin, ymin, width, height = ann['bbox']
        xmax = xmin + width
        ymax = ymin + height
        
        # 使用类别名称代替 category_id
        category_name = categories.get(ann['category_id'], str(ann['category_id']))
        
        labelme_json['shapes'].append({
            "label": category_name,  # 使用类别名称
            "points": [
                [xmin, ymin],  # 左上角
                [xmax, ymax]   # 右下角
            ],
            "group_id": None,
            "description": "",
            "shape_type": "rectangle",  # 矩形框
            "flags": {},
            "mask": None  # 目标检测中没有掩膜
        })

        """ 这样就可以把目标检测的数据集拿去做分割，不会有sam2的误分割
        labelme_json['shapes'].append({
            "label": category_name,  # 使用类别名称，而不是类别ID
            "points": [
                [xmin, ymin],
                [xmax, ymin],
                [xmax, ymax],
                [xmin, ymax]
            ],
            "shape_type": "polygon",  # 如果你使用的是矩形框，可以改成 "rectangle"
            "flags": {}
        })  
        """

    # 保存为单个图片的 JSON 文件
    output_json_path = os.path.join(output_dir, f'{img_name.split(".")[0]}.json')
    with open(output_json_path, 'w') as f_out:
        json.dump(labelme_json, f_out, indent=4)

print(f"转换完成，所有文件已保存到：{output_dir}")


转换完成，所有文件已保存到：/root/dataset/converted_labelme_json/


In [51]:
import json
import os
import numpy as np
from pycocotools import mask
from shutil import copyfile

# 输入输出路径
input_json = '/root/project/research/Yolo/yoloe/custom_train_segm.json'
output_dir = '/root/dataset/converted_labelme_seg_json/'

# 读取 COCO 格式的标注 JSON 文件
with open(input_json, 'r') as f:
    data = json.load(f)

# 检查输出目录是否存在
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 创建类别映射，将 category_id 映射到 category_name
categories = {category['id']: category['name'] for category in data['categories']}

# COCO 格式中的 'images' 和 'annotations' 数据
images = data['images']
annotations = data['annotations']

# 为每张图片生成对应的 LabelMe JSON 文件
for img_info in images:
    img_name = img_info['file_name']
    img_id = img_info['id']
    
    # 查找该图片的分割标注
    img_annotations = [ann for ann in annotations if ann['image_id'] == img_id]

    # 创建 LabelMe 格式的 JSON
    labelme_json = {
        "version": "4.5.7",  # 可以根据实际版本修改
        "flags": {},
        "shapes": [],
        "imagePath": img_name,
        "imageData": None,
        "imageHeight": img_info['height'],
        "imageWidth": img_info['width']
    }

    # 转换 COCO 格式的分割为 LabelMe 支持的多边形格式
    for ann in img_annotations:
        # COCO 格式中的 'segmentation' 是一个多边形或 RLE
        segmentation = ann['segmentation']
        
        if isinstance(segmentation, list):  # 多边形分割
            for poly in segmentation:
                # 使用类别名称代替 category_id
                category_name = categories.get(ann['category_id'], str(ann['category_id']))
                labelme_json['shapes'].append({
                    "label": category_name,  # 使用类别名称
                    "points": np.array(poly).reshape(-1, 2).tolist(),
                    "shape_type": "polygon",  # 如果你想使用矩形，改成 "rectangle"
                    "flags": {}
                })
        elif isinstance(segmentation, dict):  # RLE 分割
            rle = segmentation
            # 解码 RLE
            mask_array = mask.decode(rle)
            # 获取分割区域的轮廓点
            contours = find_contours(mask_array)
            for contour in contours:
                # 使用类别名称代替 category_id
                category_name = categories.get(ann['category_id'], str(ann['category_id']))
                labelme_json['shapes'].append({
                    "label": category_name,  # 使用类别名称
                    "points": contour.tolist(),
                    "shape_type": "polygon",
                    "flags": {}
                })

    # 保存为单个图片的 JSON 文件
    output_json_path = os.path.join(output_dir, f'{img_name.split(".")[0]}.json')
    with open(output_json_path, 'w') as f_out:
        json.dump(labelme_json, f_out, indent=4)

print(f"转换完成，所有文件已保存到：{output_dir}")

def find_contours(mask_array):
    """
    使用 OpenCV 查找二值化掩膜中的轮廓。
    """
    import cv2
    contours, _ = cv2.findContours(mask_array.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return [contour.reshape(-1, 2) for contour in contours]


转换完成，所有文件已保存到：/root/dataset/converted_labelme_seg_json/
